## Essential Libraries for Data Science and Machine Learning

Essential Python libraries is imported, each tailored for specific roles within data science and machine learning workflows.

In [220]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import pickle

In [270]:
ticker = ['MSFT', 'UPS']
start_date = '2014-03-15'
end_date = '2023-03-15'

## Step 1: Loading Financial Data

The Read_Data function serves as a step for loading financial data into a Python analysis environment. It automates the retrieval of historical data for a specified financial instrument, identified by its ticker, over a given date range from a CSV file. By constructing a file path based on the ticker symbol, start date, and end date, it locates the correct CSV file within a predetermined directory (Dfs). The data is then loaded into a pandas DataFrame, providing a structured format suitable for subsequent data analysis and manipulation steps.

In [271]:
def Read_Data(ticker, start_date, end_date):

    """
    Reads data from a CSV file into a pandas DataFrame.
    
    This function constructs a file path from a specified ticker symbol, start date, and end date,
    then reads the corresponding CSV file from the 'Dfs' folder within the current working directory.
    
    Parameters:
    - ticker (str): The ticker symbol for the data file.
    - start_date (str): The start date of the data range.
    - end_date (str): The end date of the data range.
    
    Returns:
    - pd.DataFrame: A DataFrame containing the data from the CSV file.
    
    """
    
    # Name of the folder where your CSV file is located
    folder_name = 'Dfs'
    
    # Construct the file name using the provided parameters.
    file_name = f"{ticker}_{start_date}_{end_date}.csv"
    
    # Retrieve the current working directory to build the full file path.
    current_directory = os.getcwd()
    
    # Construct the full path to the CSV file
    file_path = os.path.join(current_directory, folder_name, file_name)
    
    # Read and return the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    return df

## Step 2: Preparing Data for Analysis

The Finalize_df function is a step for preparing the loaded financial data for further analysis. It focuses on cleaning the data by performing two main tasks: removing unnecessary columns related to target predictions that do not match the specified analysis interval, and eliminating any rows that contain missing values (NaN). This process ensures the integrity and relevance of the data. Additionally, it identifies the column that will be used for target predictions, aligning with the chosen interval, making the data set ready for predictive modeling or any subsequent analysis tasks.

In [272]:
def Finalize_df(df, ticker, start_date, end_date, interval):

    """
    Cleans the DataFrame by dropping specific columns and rows with NaN values, and identifies the target column.
    
    This function iterates through the DataFrame's columns to remove any that contain the word 'Target' 
    but do not match the specified interval. It then identifies the remaining 'Target' column and 
    removes any rows in the DataFrame that contain NaN values, ensuring data integrity for further analysis.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame to be cleaned and finalized.
    - interval (int): The interval value used to identify the relevant 'Target' column to retain.
    
    Returns:
    - tuple: A tuple containing:
        - The name of the target column (str).
        - The cleaned DataFrame (pd.DataFrame) with irrelevant target columns removed and no NaN values.
    """
    
    # Identify columns to drop: those including 'Target' in their name but not matching the specified interval.
    columns_to_drop = [col for col in df.columns if 'Target' in col and 'Target_'+str(interval) != col]
    
    # Drop identified columns from the DataFrame.
    dft = df.drop(columns=columns_to_drop)
    
    # Extract the name of the remaining 'Target' column.
    Target = [col for col in dft.columns if 'Target' in col][0]

    # Get the last row of the DataFrame
    last_row = dft.tail(1)
    del last_row[Target]
    try:
        # Save the last row to a CSV file for future predictions
        last_row.to_csv(f"Last_Rows/last_row_of_{ticker}_{start_date}_{end_date}.csv", index=False)

    except FileNotFoundError:
        # Handle the case where the 'Last_Rows' directory does not exist.
        print("Error: The specified directory 'Last_Rows' does not exist.")
    
    # Remove rows with any NaN values across all columns to ensure data completeness.
    df_cleaned = dft.dropna()

    return Target, df_cleaned

## Step 3: Segmenting Data for Model Training and Testing

The Split_train_test function is crucial for dividing the cleaned dataset into appropriate subsets for training and testing machine learning models. It first segregates the dataset into features (independent variables) and the target (dependent variable) based on the column names. It excludes non-feature columns like 'Date' and the specified target column from the features set. Utilizing a predefined proportion (e.g., 80% training, 20% testing) and a fixed random state to ensure the splits are reproducible, it then divides these datasets into training and testing sets. This step is necessary for evaluating the performance of predictive models in a controlled and consistent manner, helping to ensure that the model can generalize well to unseen data.

In [273]:
def Split_train_test(df, Target):

    """
    Splits the DataFrame into training and testing datasets for features and target.

    The function separates the DataFrame into features (X) and target (y) datasets by dropping 
    the specified target column and any non-feature columns (e.g., 'Date'). It then splits these 
    datasets into training and testing sets using a predefined test size and random state to ensure 
    reproducibility.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the dataset to be split.
    - Target (str): The name of the target column in the DataFrame.
    
    Returns:
    - tuple: Contains the training and testing sets for both features (X_train, X_test) and 
      target (y_train, y_test).
    """
    
    # Prepare the features dataset by dropping the target column and other non-numeric/non-feature columns.
    X = df.drop([Target, 'Date'], axis=1)
    # Extract the target column as the target dataset.
    y = df[Target]
    
     # Split the data into training and testing sets, allocating 20% of the data to the testing set and
    # setting a random state for reproducibility.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

## Step 4: Building and Evaluating
### The Linear Regression Model

The Model_with_LR function marks a essential step in the project: the construction and evaluation of a Linear Regression model. After preparing the data, this function leverages the training set to train a Linear Regression model, applying it thereafter to the testing set to forecast outcomes. The function assesses the model's accuracy and efficiency by computing two key metrics: the Mean Squared Error (MSE) and the R-squared (R²) score. These metrics provide insights into the model's predictive power and the variance it explains, respectively, thereby enabling a quantitative evaluation of the model's performance in predicting the target variable based on the features.

In [274]:
def Model_with_LR(X_train, X_test, y_train, y_test):

    """
    Initializes, fits, and evaluates a Linear Regression model using the provided training and testing datasets.
    
    This function creates a Linear Regression model, fits it to the training data, and then uses the model
    to make predictions on the testing set. It evaluates the performance of the model by calculating the
    mean squared error (MSE) and the R-squared (R2) score between the actual and predicted values for the target.
    
    Parameters:
    - X_train (pd.DataFrame): The training dataset containing the features.
    - X_test (pd.DataFrame): The testing dataset containing the features.
    - y_train (pd.Series): The training dataset containing the target variable.
    - y_test (pd.Series): The testing dataset containing the target variable.
    
    Returns:
    - tuple: Contains the fitted Linear Regression model, the MSE, and the R2 score.
    """
    
    # Initialize the Linear Regression model.
    LR = LinearRegression()
    # Fit the model to the training data.
    LR.fit(X_train, y_train)
    
    # Use the fitted model to make predictions on the test dataset.
    y_pred = LR.predict(X_test)
    
    # Evaluate the model's performance by calculating the Mean Squared Error and R-squared score.
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return LR, mse, r2

### The OLS Regression Model

The Model_with_OLS function implements an Ordinary Least Squares (OLS) regression analysis, following the preparation and segmentation of the dataset. It adds a constant to the feature sets to include an intercept in the model, fits the OLS model to the training data, and then assesses its performance on the test data. The evaluation is carried out through the mean squared error (MSE) and R-squared (R²) score, offering insights into the model's predictive accuracy and the proportion of variance in the dependent variable explained by the independent variables. This step is crucial for understanding how well the linear relationship has been modeled and for identifying potential areas for model improvement or refinement.

In [275]:
def Model_with_OLS(X_train, X_test, y_train, y_test):

    """
    Initializes, fits, and evaluates an Ordinary Least Squares (OLS) regression model.
    
    This function extends the X_train and X_test datasets with a constant term to account for the intercept 
    in the OLS model, fits the model to the training data, and evaluates its performance on the testing set 
    by calculating both the mean squared error (MSE) and R-squared (R2) score.
    
    Parameters:
    - X_train (pd.DataFrame): Training dataset containing the features.
    - X_test (pd.DataFrame): Testing dataset containing the features.
    - y_train (pd.Series): Training dataset containing the target variable.
    - y_test (pd.Series): Testing dataset containing the target variable.
    
    Returns:
    - tuple: Contains the fitted OLS model, the MSE, and the R2 score of the model on the testing set.
    """
    
    # Extend the features matrix of the training set with a column for the constant term.
    X_with_const = sm.add_constant(X_train)
    
    # Fit the OLS (Ordinary Least Squares) model
    ols = sm.OLS(y_train, X_with_const).fit()
    
    # Extend the features matrix of the testing set with a column for the constant term.
    X_test_with_const = sm.add_constant(X_test)
    # Use the fitted model to make predictions on the extended testing set.
    y_pred = ols.predict(X_test_with_const)
    
    # Evaluate the performance of the model on the testing set.
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return ols, mse, r2

### The XGBoost Regression Model

The Model_with_XGB function deploys an XGBoost regression model to the project. After preparing the dataset, this function configures an XGBoost model with pre-defined hyperparameters, trains it on the dataset, and evaluates its predictive performance using the test data. The function focuses on understanding the model's accuracy through mean squared error (MSE) and R-squared (R²) metrics, crucial for assessing the model's effectiveness in capturing the complexity of the data and making accurate predictions.

In [276]:
def Model_with_XGB(X_train, X_test, y_train, y_test):

    """
    Initializes, fits, and evaluates an XGBoost regression model.
    
    This function creates an XGBoost regression model with specified hyperparameters, fits it to the 
    training data, and uses it to make predictions on the testing set. It evaluates the model's performance
    by calculating the mean squared error (MSE) and R-squared (R2) score between the actual and predicted
    values for the target variable.
    
    Parameters:
    - X_train (pd.DataFrame): The training dataset containing the features.
    - X_test (pd.DataFrame): The testing dataset containing the features.
    - y_train (pd.Series): The training dataset containing the target variable.
    - y_test (pd.Series): The testing dataset containing the target variable.
    
    Returns:
    - tuple: Contains the fitted XGBoost model, the MSE, and the R2 score.
    """
    
    # Initialize the XGBoost regressor model with specified hyperparameters.
    xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

    # Fit the XGBoost model to the training data.
    xgb_model.fit(X_train, y_train)
    
    # Use the trained model to make predictions on the test dataset.
    xgb_pred = xgb_model.predict(X_test)
    
    # Evaluate the model's performance on the test data using Mean Squared Error and R-squared metrics.
    xgb_mse = mean_squared_error(y_test, xgb_pred)
    xgb_r2 = r2_score(y_test, xgb_pred)
    
    return xgb_model, xgb_mse, xgb_r2

### The Random Forest Regression Model

The Model_with_RF function incorporates the use of a Random Forest regression model into the project, marking a move towards leveraging ensemble learning techniques for enhanced predictive modeling. By initiating a Random Forest model with its default settings, the function trains the model on the dataset prepared in previous steps and assesses its predictive power on the test set. The evaluation focuses on the mean squared error (MSE) and R-squared (R²) metrics to gauge the model's accuracy and the proportion of variance it explains in the target variable.

In [277]:
def Model_with_RF(X_train, X_test, y_train, y_test):

    """
    Initializes, fits, and evaluates a Random Forest regression model.
    
    This function creates a Random Forest regression model with default hyperparameters, fits it to the
    training data, and uses it to make predictions on the testing set. It evaluates the model's performance
    by calculating the mean squared error (MSE) and R-squared (R2) score between the actual and predicted
    values for the target variable.
    
    Parameters:
    - X_train (pd.DataFrame): The training dataset containing the features.
    - X_test (pd.DataFrame): The testing dataset containing the features.
    - y_train (pd.Series): The training dataset containing the target variable.
    - y_test (pd.Series): The testing dataset containing the target variable.
    
    Returns:
    - tuple: Contains the fitted Random Forest model, the MSE, and the R2 score.
    """
    
    # Initialize and fit the Random Forest regressor model to the training data.
    rf_model = RandomForestRegressor()
    rf_model.fit(X_train, y_train)
    
    # Use the trained model to make predictions on the test dataset.
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluate the model's performance on the test data using Mean Squared Error and R-squared metrics.
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    
    return rf_model, mse_rf, r2_rf

## Step 5: Selecting and Saving the Optimal Model

The Save_Best_Model function integrates model comparison and selection, targeting the identification of the most effective regression model for the dataset based on its predictive performance. This function systematically evaluates Linear Regression, OLS, XGBoost, and Random Forest models, all previously prepared and assessed, to determine the best model according to mean squared error (MSE) and R-squared (R²) metrics. Priority is given to minimizing MSE while using R² as a secondary criterion. Only models surpassing a high threshold of predictive accuracy (R² >= 0.90) are considered for final selection. The best-performing model, if meeting the set criteria, is then serialized (pickled) for persistence, facilitating its reuse without retraining. This step encapsulates the project's goal of optimizing predictive accuracy while providing a mechanism for efficiently deploying the chosen model in future applications.

In [278]:
def Save_Best_Model(X_train, X_test, y_train, y_test):

    """
    Evaluates multiple regression models on the given dataset, identifies the best performing model based on
    mean squared error (MSE) and R-squared (R2) metrics, and pickles the best model if its R2 score is above 0.90.
    
    The function considers Linear Regression, OLS, XGBoost, and Random Forest models, comparing their performance
    on the given training and testing datasets. The best model is determined by the lowest MSE, with a tiebreaker
    based on the highest R2 score.
    
    Parameters:
    - X_train (pd.DataFrame): The training dataset containing the features.
    - X_test (pd.DataFrame): The testing dataset containing the features.
    - y_train (pd.Series): The training dataset containing the target variable.
    - y_test (pd.Series): The testing dataset containing the target variable.
    
    Returns:
    - The tuple of the best model, its MSE, and R2 score, if its R2 score is above 0.90; otherwise, None.
    """
    
    # Initialize variables to store the best model and its metrics
    best_model = None
    best_mse = float('inf')  # Initialize with a large value
    best_r2 = -float('inf')   # Initialize with a small value

    # Generate models using predefined functions.
    Model_LR = Model_with_LR(X_train, X_test, y_train, y_test)
    Model_OLS = Model_with_OLS(X_train, X_test, y_train, y_test)
    Model_XGB = Model_with_XGB(X_train, X_test, y_train, y_test)
    Model_RF = Model_with_RF(X_train, X_test, y_train, y_test)

    # List of models to evaluate
    models = {
        "Linear Regression": Model_LR,
        "OLS": Model_OLS,
        "XGBoost": Model_XGB,
        "Random Forest": Model_RF
    }

    # Evaluate each model, updating the best model based on MSE and R2.
    for model_name, (model, mse, r2) in models.items():

        # Check if the current model has better performance than the best model found so far
        if mse < best_mse:
            best_model = model_name
            best_mse = mse
            best_r2 = r2

    # Conditionally choose the best model if its performance is sufficiently high.
    if best_r2>=0.90:
        
        if best_model == "Linear Regression":
            model_to_pickle = Model_LR
        elif best_model == "OLS":
            model_to_pickle = Model_OLS
        elif best_model == "XGBoost":
            model_to_pickle = Model_XGB
        elif best_model == "Random Forest":
            model_to_pickle = Model_RF

    else:
        print('Could not create model with high coverage (R^2 >= 0.9) for the given data. Consider changing input parameters.')
        
    return model_to_pickle

## Step 6: Persisting the Best Model for Future Use

The Make_Pickle function encapsulates the final step in the predictive modeling process: saving the optimal model for future application. By serializing the chosen model into a binary file using Python's pickle module, this function ensures that the model can be easily and efficiently reloaded for subsequent predictions, without the need to retrain. The naming convention for the output file incorporates both the ticker symbol and the interval for which the model was developed, facilitating straightforward identification and retrieval of the model. This step is crucial for operationalizing the model in practical financial analysis scenarios, allowing for quick deployment in predictive tasks related to the specified financial instrument and analysis interval.

In [279]:
def Make_Pickle(model_to_pickle, interval, ticker, start_date, end_date):

    """
    Pickles and saves the provided model to a file, named according to the specified ticker and interval.
    
    The function writes the model to a binary file within the 'Pickles' directory. The naming convention for the file
    includes the model's associated ticker symbol and interval, ensuring easy identification and retrieval of the model
    for future predictions or analysis.
    
    Parameters:
    - model_to_pickle: The model object to be pickled and saved.
    - interval (str or int): The interval associated with the model, contributing to the file's name.
    - ticker (str): The ticker symbol associated with the model, contributing to the file's name.
    
    Outputs:
    - On success, the model is saved to a .pkl file in the 'Pickles' directory, and a confirmation message is printed.
    - On failure, an error message is printed detailing the nature of the problem encountered.

    Raises:
    - FileNotFoundError: If the 'Pickles' directory does not exist.
    - IOError: If there is an issue writing to the file.
    - pickle.PicklingError: If the model object cannot be serialized.
    - Exception: Catches any other unexpected errors that occur during execution.
    
    """
    # Open the file and pickle the model object to the file.
    try:
        with open(f"Pickles/best_model_{ticker}_{interval}_{start_date}_{end_date}.pkl", "wb") as f:
            # Serialize and save the 'model_to_pickle' object into the file.
            pickle.dump(model_to_pickle, f)
        # Print a success message indicating the model has been successfully pickled and saved.    
        print(f"Model pickled successfully for {ticker}_{interval}.")
    except FileNotFoundError:
        # Handle the case where the 'Pickles' directory does not exist.
        print("Error: The specified directory 'Pickles' does not exist.")
    except IOError:
        # Handle general input/output errors, such as failure in file writing.
        print("IOError: There was an issue writing to the file.")
    except pickle.PicklingError:
        # Handle errors specific to the pickling process, such as if the object cannot be serialized.
        print("PicklingError: There was an issue pickling the model.")
    except Exception as e:
        # Catch any other unexpected exceptions and print an error message detailing the problem.
        print(f"An unexpected error occurred: {e}")


## Step 7: Automating Model Development and Persistence Across Time Intervals

The Make_Predictions function has a comprehensive process tailored for financial data analysis, specifically aimed at generating and saving predictive models for a range of time intervals. This automated pipeline initiates with reading historical stock data for a specified ticker within a given date range. It then iteratively cleans the data, selects the best model based on predefined performance metrics (mean squared error and R-squared score), and finally serializes the optimal model for each interval, making it readily accessible for future predictions.

For each interval in the predetermined list, the function executes several key steps: data preparation (cleaning and finalizing), dataset partitioning into training and testing sets, model comparison and selection, and model serialization. This end-to-end process not only enhances efficiency in model development but also ensures that the models are tailored to specific intervals, thereby improving the relevance and accuracy of predictions for different time frames. 

In [280]:
def Make_Predictions(ticker, start_date, end_date):

    """
    Processes stock data for a given ticker and date range to train, evaluate, and save the best predictive model
    for various time intervals.
    
    For each specified interval, this function reads historical stock data, cleans it, splits it into training and testing
    sets, identifies the best model based on performance metrics, and pickles this model for future use.
    
    Parameters:
    - ticker (str): The stock ticker symbol for which data is processed.
    - start_date (str): The start date for the data range to be processed.
    - end_date (str): The end date for the data range to be processed.
    
    Processes:
    - For each interval in a predefined list, performs data cleaning, model training and evaluation,
      and pickles the best-performing model.
    
    Outputs:
    - Saves pickled models for each interval to the disk.
    """

    # Define the list of intervals to generate models for.
    List_of_intervals = [1, 2, 7, 14, 28]

    # Read the data for the specified ticker and date range.
    df = Read_Data(ticker, start_date, end_date)
    print(ticker,'\'s Data frame has been built Successfully!')
    
    # Iterate over each interval, process the data, and save the best model.
    for interval in List_of_intervals: 
        # Prepare the data for the given interval.
        Target, df_cleaned = Finalize_df(df, ticker, start_date, end_date, interval)
        # Split the data into training and testing sets.
        X_train, X_test, y_train, y_test = Split_train_test(df_cleaned, Target)
        # Identify and save the best model for the interval.
        model_to_pickle = Save_Best_Model(X_train, X_test, y_train, y_test)
        # Check if a model was selected to be pickled before attempting to pickle it.
        if model_to_pickle:
            print(f'Best model selected for, {interval}, with MSE = {best_mse} and Rsquared = {best_r2}')
            Make_Pickle(model_to_pickle, interval, ticker, start_date, end_date)
        else:
            print(f"No suitable model found for interval {interval} to pickle.")

In [281]:
for t in ticker:
    Make_Predictions(t, start_date, end_date)

Model pickled successfully for MSFT_1.
Model pickled successfully for MSFT_2.
Model pickled successfully for MSFT_7.
Model pickled successfully for MSFT_14.
Model pickled successfully for MSFT_28.
Model pickled successfully for UPS_1.
Model pickled successfully for UPS_2.
Model pickled successfully for UPS_7.
Model pickled successfully for UPS_14.
Model pickled successfully for UPS_28.
